**Installing latest pickle to unserialize our data from database**

In [2]:
!pip install pickle5


     |████████████████████████████████| 133kB 22.0MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp36-cp36m-linux_x86_64.whl size=218612 sha256=d8e91e2f07fcafff53633d6c2dcbf19cffba283b29ac9d01cf6ee06a06e4f77e
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


**Adding needed imports**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
import  pickle5 as pickle
from HandReading import HandReading
from Imu import Imu
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, LSTM
from sklearn.metrics import accuracy_score as acc
from tensorflow.keras.utils import plot_model
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping

**Creating dictionary mappers**


In [ ]:
dict_word_id = {}
dict_id_word = {}

list_word = os.listdir('/content/drive/MyDrive/database') 
for j in range(len(list_word)) : 
  dict_word_id[list_word[j]] = j
  dict_id_word[j] = list_word[j]
print(dict_word_id)
print(dict_id_word)

**Preparing input and labels data**

In [ ]:
input_data, output = [], []

list_word = os.listdir('/content/drive/MyDrive/database') 

for word in list_word:
  list_pickle_file = os.listdir(f'/content/drive/MyDrive/database/{word}')
  k = 0 
  for pickle_file in list_pickle_file:
    k+=1
    with open(f'/content/drive/MyDrive/database/{word}/{pickle_file}', 'rb') as input:
      readings = pickle.load(input)
      for reading in readings:
        finger = []
        for i in range(5):
          finger.append(np.float16(reading.imus[i].accel[0]/255.0))
          finger.append(np.float16(reading.imus[i].accel[1]/255.0))
          finger.append(np.float16(reading.imus[i].accel[2]/255.0))
          finger.append(np.float16(reading.imus[i].gyro[0]/255.0))
          finger.append(np.float16(reading.imus[i].gyro[1]/255.0))
          finger.append(np.float16(reading.imus[i].gyro[2]/255.0))
          pass
      
        input_data.append(finger)
        
        out = np.zeros(11)
        out[dict_word_id[word]] = 1.0
        output.append(out)

input_data = np.array(input_data)
input_data = input_data.reshape((input_data.shape[0], input_data.shape[1], 1))
output = np.array(output)

**Creating neural network model**

In [ ]:
model = Sequential()

model.add(LSTM(units=50, input_shape=input_data[0].shape, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

model.add(Dense(11, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

**Separate data and train**

In [ ]:
from sklearn.model_selection import StratifiedKFold
kfoldK = 10
kfold = StratifiedKFold(n_splits=10, shuffle=True)
callback = EarlyStopping(monitor='val_accuracy', min_delta = 0.001, patience=3)
iterations = 0 
sumOfAccuracy = 0
maxepoch = 100
for train, test in kfold.split(input_data, output.argmax(1)):
  iterations += 1
  trainX, testX = input_data[train], input_data[test]
  trainY, testY = output[train], output[test]

  trainX, validX, trainY, validY = train_test_split(
    trainX, trainY, test_size=0.3, stratify=trainY, random_state=42)
  
  model.fit(trainX, trainY, epochs=maxepoch, batch_size=32, verbose=2, callbacks=[callback], validation_data=(validX, validY))
  predicted = model.predict(testX)
  pred = []
  for j in predicted:
    arg = np.argmax(j)
    pred.append(arg)
  pred = np.array(pred)
  y_one = []
  for y in testY :
    arg = np.argmax(y)
    y_one.append(arg)
  y_one = np.array(y_one)
  sumOfAccuracy += acc(y_one,pred)
  avgAccuray = sumOfAccuracy/iterations
  print(f"average accuracy = {avgAccuray}")

**Analyse accuracies**


In [ ]:
prd = model.predict(testX)
pred = []
for j in range(len(prd)):
  lis = prd[j]
  arg = np.argmax(lis)
  pred.append(arg)
pred = np.array(pred)

y_one = []
for y in testY :
  arg = np.argmax(y)
  y_one.append(arg)
y_one = np.array(y_one)

print(classification_report(y_one, pred))
print(dict_id_word)
print()
print(acc(y_one,pred))
print(f"average accuracy = {sumOfAccuracy/iterations}")

**Real life usage example**

In [ ]:
with open(f'/content/drive/MyDrive/database/prazer-em-te-conhecer/3.pkl', 'rb') as input:
  readings = pickle.load(input)
  input_test_data = []
  j = 0
  for reading in readings:
    finger = []
    for i in range(5):
      finger.append(np.float16(reading.imus[i].accel[0]/255.0))
      finger.append(np.float16(reading.imus[i].accel[1]/255.0))
      finger.append(np.float16(reading.imus[i].accel[2]/255.0))
      finger.append(np.float16(reading.imus[i].gyro[0]/255.0))
      finger.append(np.float16(reading.imus[i].gyro[1]/255.0))
      finger.append(np.float16(reading.imus[i].gyro[2]/255.0))
    
  
    input_test_data.append(finger)
    
input_test_data = np.array(input_test_data)
input_test_data = input_test_data.reshape((input_test_data.shape[0], input_test_data.shape[1], 1))

for inputData in input_test_data:
  result1 = model.predict(inputData.reshape(1,30,1))
  print(dict_id_word[np.argmax(result1)])

**Plot model structure**

In [ ]:
plot_model(model, to_file='topology.png',show_shapes=True)

In [ ]:
model.save(f"deep-lstm-{maxepoch}maxepoch-k{kfoldK}fold-{avgAccuray}avgacc.h5", save_format='.h5')
model.save(f"deep-lstm-{maxepoch}maxepoch-k{kfoldK}fold-{avgAccuray}avgacc")


In [ ]:
cm = confusion_matrix(y_one, pred)
print(cm)
plt.rcParams["figure.figsize"] = (20,8)
plt.imshow(cm, cmap=plt.cm.Reds)
plt.xlabel("Sinais obtidos")
plt.ylabel("Sinais corretos")
tick_marks = np.arange(len(list(dict_word_id.keys())[:-1]))
plt.tick_params(axis='both', which='major', labelsize=12)
plt.xticks(tick_marks, list(dict_word_id.keys())[:-1], rotation=60)
plt.yticks(tick_marks, list(dict_word_id.keys())[:-1])
plt.title('Matriz de Confusão')
plt.colorbar()
plt.show()